Run basic ResNet evaluation on subset of COCO

In [15]:
import os
os.chdir('/home/jknize/main/repo/CSC578/detectron2')

1. Load data

In [22]:
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data.datasets import load_coco_json
from pycocotools.coco import COCO
import json

In [23]:
annotation_path = "../datasets/coco/annotations/instances_val2017.json"
image_dir = "../datasets/coco_cls"
output_path = "../datasets/coco/annotations/filtered_annotations.json"

# Load COCO annotations
coco = COCO(annotation_path)

# Specify the classes
desired_classes = ["book", "bottle", "chair", "dog", "person"]
class_ids = coco.getCatIds(catNms=desired_classes)

# Filter images: get the list of image filenames in the filtered image directory
filtered_image_filenames = set(os.listdir(image_dir))

# Initialize the filtered annotations dictionary
filtered_annotations = {
    "info": coco.dataset["info"],
    "licenses": coco.dataset["licenses"],
    "images": [],
    "annotations": [],
    "categories": [cat for cat in coco.loadCats(class_ids)]
}

# Get image IDs that match your filtered images
image_ids = set()
for class_id in class_ids:
    img_ids_for_class = coco.getImgIds(catIds=[class_id])
    for img_id in img_ids_for_class:
        img_info = coco.loadImgs([img_id])[0]
        img_filename = img_info['file_name']
        if img_filename in filtered_image_filenames:
            image_ids.add(img_id)

# Add the filtered images to the result
filtered_annotations["images"] = coco.loadImgs(list(image_ids))

# Add the annotations that correspond to the filtered images and classes
for img_id in image_ids:
    ann_ids = coco.getAnnIds(imgIds=[img_id], catIds=class_ids, iscrowd=None)
    filtered_annotations["annotations"].extend(coco.loadAnns(ann_ids))

# Save the filtered annotations to a new JSON file
with open(output_path, "w") as f:
    json.dump(filtered_annotations, f)

loading annotations into memory...
Done (t=0.48s)
creating index...
index created!


In [24]:
DatasetCatalog.remove("coco_subset")
MetadataCatalog.remove("coco_subset")
DatasetCatalog.register("coco_subset", lambda: load_coco_json(output_path, image_dir))
MetadataCatalog.get("coco_subset").set(thing_classes=["book", "bottle", "chair", "dog", "person"])

Metadata(name='coco_subset', thing_classes=['book', 'bottle', 'chair', 'dog', 'person'])

2. Load Model

In [25]:
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor

In [26]:
# load simple ResNet-50 model without FPN
cfg = get_cfg()
cfg.merge_from_file("configs/COCO-Detection/faster_rcnn_R_50_C4_1x.yaml")
# use COCO-pretrained detection weights for quick evaluation
cfg.MODEL.WEIGHTS = "detectron2://COCO-Detection/faster_rcnn_R_50_C4_1x/137849393/model_final_721ade.pkl"
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5

cfg.DATASETS.TRAIN = ("coco_subset",)
cfg.DATASETS.TEST = ("coco_subset",)

Evaluate model

In [28]:
# from Google Colab tutorial
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("coco_subset", output_dir="./knize/output")
val_loader = build_detection_test_loader(cfg, "coco_subset")
print(inference_on_dataset(predictor.model, val_loader, evaluator))

Using previously cached COCO format annotations at './knize/output/coco_subset_coco_format.json'. You need to clear the cache file if your dataset has been modified.


AssertionError: Dataset 'coco_subset' is empty!

In [30]:
coco = COCO("../datasets/coco/annotations/filtered_instances_val2017.json")
anns = coco.loadAnns(coco.getAnnIds())
print([ann['category_id'] for ann in anns])

loading annotations into memory...
Done (t=0.16s)
creating index...
index created!
[18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1